In [4]:
import pandas as pd
import os
from influxdb_client import InfluxDBClient
# from featurestoresdk.feature_store_sdk import FeatureStoreSdk
from sodyn99.feature_store_sdk import FeatureStoreSdk

INFLUXDB_HOST = os.getenv('INFLUXDB_HOST')
INFLUXDB_PORT = os.getenv('INFLUXDB_PORT')
INFLUXDB_TOKEN = os.getenv('INFLUXDB_TOKEN')
INFLUXDB_ORG = "primary"
INFLUXDB_BUCKET = "UAVData"

TRAINING_JOB_NAME = "uavtest"
FEATURE_NAME = "x"

In [5]:
fs_sdk = FeatureStoreSdk()

sdk = fs_sdk.get_features(TRAINING_JOB_NAME, [FEATURE_NAME])

print("\033[92mData from Cassandra:\033[0m\n", sdk)

Data from Cassandra:
                           x
0       -2.7278003677876597
1       -2.7044567059621523
2       -2.6829307181061064
3       -2.6607717007217073
4       -2.6363856183323264
...                     ...
249995  0.20020159056237843
249996  0.19776726125967284
249997  0.19755735498886612
249998   0.1966440326878903
249999  0.19841127818046236

[250000 rows x 1 columns]


In [6]:
client = InfluxDBClient(url=f"http://{INFLUXDB_HOST}:{INFLUXDB_PORT}", token=INFLUXDB_TOKEN, org=INFLUXDB_ORG)
query_api = client.query_api()
query = f'''
from(bucket:"{INFLUXDB_BUCKET}") |> range(start: -1000d, stop: now()) |> filter(fn: (r) => r._measurement == "liveCell") |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
'''
result = query_api.query(query=query)

t = []

if not result:
    print("\033[91mNo data found.\033[0m")
else:
    for table in result:
        for record in table.records:
            t.append(record[FEATURE_NAME])
client.close()

df = pd.DataFrame(t, columns=[FEATURE_NAME])
print("\033[92mData from InfluxDB:\033[0m\n", df)

Data from InfluxDB:
                x
0      -2.727800
1      -2.704457
2      -2.682931
3      -2.660772
4      -2.636386
...          ...
249995  0.200202
249996  0.197767
249997  0.197557
249998  0.196644
249999  0.198411

[250000 rows x 1 columns]
